# Tarea 1

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
import pandas as pd
from enum import Enum

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

## `Qcf.time_series`

In [4]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114
fecha3 = Qcf.QCDate(15, 1, 1969)
ts[fecha3] = 19690115

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)

for d in dl:
    
    print(ts[d])
    
    if ts[d]==ts[fecha2]:
        print("listo")

ts[fecha1]: 19690113.0

19690113.0
19690114.0
listo
19690115.0


In [5]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114
fecha3 = Qcf.QCDate(15, 1, 1969)
ts[fecha3] = 19690115

print(f'ts[fecha1]: {ts[fecha1]}\n')
dl = Qcf.time_series_dates(ts)
x=fecha1.day_diff(fecha2) 
print(x)
for d in dl:
    
    print(d)
    if d==fecha2:
        print(15)
        break

ts[fecha1]: 19690113.0

1
13-1-1969
14-1-1969
15


## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

In [6]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
        
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        P=1
        fechas=Qcf.time_series_dates(fixings)#calendario con fechas hasta accrual_date
        for fecha in fechas:
            P=(1+fixings[fecha])*P
            if fecha==accrual_date:
                break
            
        dias = self.start_date.day_diff(accrual_date) #dias es fecha actual menos inicial
        accrued=(P - 1) * 360 / (dias)
        
        return accrued
        
        
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        
        dias = self.start_date.day_diff(accrual_date) #dias es fecha actual menos inicial
        acc_interest=self.notional*self.get_accrued_rate(accrual_date, fixings) * dias/360
        
        return acc_interest
       
    
    def amount(self, fixings: Qcf.time_series) -> float:
       
        valor=0
        for fecha in calendario: #fecha donde pago intereses
            valor=valor+self.get_accrued_interest(fecha, fixings)
           
        amount=nocional+valor
        return amount

Construcción de un objeto `Qcf.InterestRateIndex`.

In [7]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [8]:
ois = OisCashflow(
    Qcf.QCDate(13, 1, 2020),
    Qcf.QCDate(13, 1, 2021),
    Qcf.QCDate(13, 1, 2021),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [9]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
print(df_curva)

    plazo      tasa        df
0       1  0.000811  0.999998
1       7  0.000841  0.999984
2      14  0.000780  0.999970
3      21  0.000774  0.999955
4      33  0.000781  0.999929
5      61  0.000781  0.999870
6      92  0.000811  0.999796
7     125  0.000781  0.999733
8     152  0.000760  0.999683
9     182  0.000750  0.999626
10    212  0.000736  0.999573
11    243  0.000725  0.999518
12    273  0.000720  0.999462
13    306  0.000714  0.999402
14    335  0.000708  0.999350
15    365  0.000702  0.999298
16    547  0.000642  0.999039
17    730  0.000568  0.998865
18   1097  0.000750  0.997748
19   1462  0.001258  0.994976
20   1826  0.001939  0.990348
21   2191  0.002775  0.983483
22   2556  0.003603  0.975082
23   2924  0.004384  0.965489
24   3288  0.005117  0.954953
25   3653  0.005812  0.943495
26   4383  0.006917  0.920290
27   5479  0.008064  0.885997
28   7306  0.009253  0.830924
29   9133  0.009657  0.785338
30  10957  0.009881  0.743326
31  14610  0.009408  0.686197
32  18262 

In [10]:
try:
    ois.test1(0)
except Exception as e:
    print('Hacer algo')

Hacer algo


In [11]:
print(ois.start_date)

13-1-2020


In [12]:
print(ois.interest_rate)

0.000000 Act360 Lin


## Funciones

In [13]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    
    
    sofr1= zcc.get_discount_factor_at(val_date)
    sofr2= zcc.get_discount_factor_at(OisCashflow.end_date)
    e_rate=((sofr2/sofr1)-1)*360.0/(val_date.day_diff(OisCashflow.end_date))
    
    return e_rate
   
    # Para un OIS calculando en t0 un cashflow entre (t0 <) t1 < t2
    # ICP(t0)*wf(to, t2) = ICP(t2), ICP(t0)*w(t0, t1)=ICP(t1) -> ICP(t2)/ICP(t1)=ICP(t1,t2)
    # tasa(t1, t2) = [ICP(t1, t2) - 1]*360 / (t2-t1)
    # El ICP se puede pensar como el account asociado a la tasa interbancaria chilena,
    # que es el índice overnight chileno.
    #
    # Cuando t1 < t0 < t2 tener un poquito de ojo ... porque el cashflow ya tiene fixings.

In [14]:
def present_value(val_date: Qcf.QCDate,
                  ois_cashflow: OisCashflow,
                  zcc: Qcf.ZeroCouponCurve) -> float:
   
    fecha_vcto = OisCashflow.end_date
    monto = ois_cashflow.notional
    usd = Qcf.QCUSD()

# Se construye el depósito
    depo = Qcf.SimpleCashflow(fecha_vcto, monto, usd)
# val_date fecha de hoy
    plazo = val_date.day_diff(fecha_vcto)
    tasa_int = set_expected_rate(
        val_date,
        ois_cashflow,
        zcc,
        fixings)
    valor_presente = monto * (1 + tasa_int)**(-plazo / 365)
    
    return valor_presente

## Tests

In [15]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [16]:
amount_tol = 10000
rate_tol = .001

In [17]:
exitos = 0

In [18]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [20]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [21]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [22]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [23]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [24]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [25]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 705.49474235%


Se verificará el cálculo usando `df_fixings`.

In [26]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [27]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [28]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [29]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [30]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [31]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [32]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [33]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [34]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [35]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [36]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [37]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 704.48482921%


In [38]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.


### `OisCashflow.accrued_interest`

In [39]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 50,560,456.54


In [40]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [41]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [42]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

NameError: name 'calendario' is not defined

In [43]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [44]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

NameError: name 'amount' is not defined

In [45]:
suma_exito(amount, check_amount, 'monto')

NameError: name 'amount' is not defined

### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [46]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [47]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [48]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [49]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [51]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [52]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [53]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [54]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [55]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

### `present_value`

In [56]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois,  zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [57]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [58]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [59]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [60]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    InterestRateCurve.get_discount_factor_at(ZeroCouponCurve, QCDate)
did not match C++ signature:
    get_discount_factor_at(std::shared_ptr<wrappers::InterestRateCurveWrap> {lvalue}, long)
    get_discount_factor_at(QCode::Financial::InterestRateCurve {lvalue}, long)

In [61]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 2.0
